In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [35]:
!sudo apt-get install unzip
!unzip /content/cyberbullying_data_clean.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Archive:  /content/cyberbullying_data_clean.zip
   creating: cyberbullying_data_clean/
   creating: cyberbullying_data_clean/cyberbullying/
   creating: cyberbullying_data_clean/non_cyberbullying/
  inflating: cyberbullying_data_clean/cyberbullying/3.thumbs-down-got-o64pfdnoo3btwrc9bbzr8rezrkcehr78rzsq2uwrd4.jpg  
  inflating: cyberbullying_data_clean/cyberbullying/103.500_F_277681363_LlLhT882jThUj5FbjtnG2bGsT6BW1BnC.jpg  
  inflating: cyberbullying_data_clean/cyberbullying/395.QhUFr7-Page14M%20n1.jpg  
  inflating: cyberbullying_data_clean/cyberbullying/114.peaky-blinders-1566768238.png  
  inflating: cyberbullying_data_clean/cyberbullying/42.5b76ec6180eb35a92d8b4d7f-750-563.jpg  
  inflating: cyberbullying_data_clean/cyberbullying/292.disappointed-businesswoman-in-christm

In [36]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [87]:
import pandas as pd
import os
from tqdm import tqdm

In [80]:
import base64
import requests

# OpenAI API Key
api_key = "xxx"

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}


In [121]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def response_text_to_label(response_text):
  if 'a.' in response_text:
    return 1

  if 'b.' in response_text:
    return 0

  return -1 #uncertain

In [115]:
d = {'path': [], 'true_label': [], 'prompt': [], 'response': [], 'label': []}
df = pd.DataFrame(data=d)

In [116]:
dataset_path = 'cyberbullying_data_clean'
cyberbullying_path = dataset_path + '/cyberbullying'
non_cyberbullying_path = dataset_path + '/non_cyberbullying'

cyberbullying_images = os.listdir(cyberbullying_path)
non_cyberbullying_images = os.listdir(non_cyberbullying_path)

prompt = 'Does this image contain cyberbullying content?\na. Yes\nb. No'

for image in tqdm(cyberbullying_images):
  try:
    image_path = f'{cyberbullying_path}/{image}'
    base64_image = encode_image(image_path)

    payload = {
      "model": "gpt-4-turbo",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
              'detail': 'low'
              }
            }
          ]
        }
      ],
      "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    response_text = response.json()['choices'][0]['message']['content']
    label = response_text_to_label(response_text)

    new_row = pd.DataFrame(data = {'path': [image_path], 'true_label': [1], 'prompt': [prompt], 'response': [response_text], 'label': [label]})

    df = pd.concat([df, new_row], ignore_index=True)

  except:
    print('Error: ')
    print(image)

 68%|██████▊   | 682/1003 [31:23<29:00,  5.42s/it]

Error: 
loser_sign_loser_342.jpg


100%|██████████| 1003/1003 [46:39<00:00,  2.79s/it]


In [117]:
df.to_csv('/content/drive/MyDrive/general_prompt.csv')

In [ ]:
for image in tqdm(non_cyberbullying_images):
  try:
    image_path = f'{non_cyberbullying_path}/{image}'
    base64_image = encode_image(image_path)

    payload = {
      "model": "gpt-4-turbo",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
              'detail': 'low'
              }
            }
          ]
        }
      ],
      "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    response_text = response.json()['choices'][0]['message']['content']
    label = response_text_to_label(response_text)

    new_row = pd.DataFrame(data = {'path': [image_path], 'true_label': [1], 'prompt': [prompt], 'response': [response_text], 'label': [label]})

    df = pd.concat([df, new_row], ignore_index=True)

  except:
    print('Error: ')
    print(image)

  4%|▍         | 123/2869 [05:24<1:59:18,  2.61s/it]

In [122]:
df

,path,true_label,prompt,response,label
0,cyberbullying_data_clean/cyberbullying/414.cra...,1.0,Does this image contain cyberbullying content?...,The image provided does not contain content th...,0.0
1,cyberbullying_data_clean/cyberbullying/401.clo...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image you provided shows a person...,0.0
2,cyberbullying_data_clean/cyberbullying/15.sad-...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image appears to be a creative or...,0.0
3,cyberbullying_data_clean/cyberbullying/181.1jp...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image shows a man wearing sunglas...,0.0
4,cyberbullying_data_clean/cyberbullying/595.bea...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image does not contain any visibl...,0.0
...,...,...,...,...,...
997,cyberbullying_data_clean/cyberbullying/thumbs_...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image does not contain visible co...,0.0
998,cyberbullying_data_clean/cyberbullying/39.teen...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image shows a person making a pla...,0.0
999,cyberbullying_data_clean/cyberbullying/middle_...,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image shows a person riding a bic...,0.0
1000,cyberbullying_data_clean/cyberbullying/437.7.jpg,1.0,Does this image contain cyberbullying content?...,b. No\n\nThe image shows a person giving a thu...,0.0


In [ ]:
df.to_csv('/content/drive/MyDrive/general_prompt_uncleaned.csv')